<a href="https://colab.research.google.com/github/pks98/NLP_Reviews/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [299]:
df=pd.read_csv('reviews.csv',usecols=['Review_Title','Reviews','Rating'])
df.head()

,Review_Title,Reviews,Rating
0,AWESOME...IN LOVE WITH IT !,Really impressed with the fastest delivery of ...,5.0
1,Enjoy while it lasts,This is the second time I am writing this revi...,2.0
2,Good,1. Quite good look2. Better noise cancellation...,4.0
3,not great,its a good set of ear buds but1. the quality o...,1.0
4,Best buds in the budget with Type C,I have been researching about getting a nice T...,5.0


In [300]:
df.shape

(3620, 3)

In [301]:
def data_clean(text):
  if text=='The media could not be loaded.':
    return " "
  else:
    return text

In [302]:
df['Reviews']=df['Reviews'].apply(data_clean)

In [303]:
df.fillna('',inplace=True)

In [304]:
df['text']=df['Review_Title']+' '+df['Reviews']

In [305]:
df.head()

,Review_Title,Reviews,Rating,text
0,AWESOME...IN LOVE WITH IT !,Really impressed with the fastest delivery of ...,5.0,AWESOME...IN LOVE WITH IT ! Really impressed w...
1,Enjoy while it lasts,This is the second time I am writing this revi...,2.0,Enjoy while it lasts This is the second time I...
2,Good,1. Quite good look2. Better noise cancellation...,4.0,Good 1. Quite good look2. Better noise cancell...
3,not great,its a good set of ear buds but1. the quality o...,1.0,not great its a good set of ear buds but1. the...
4,Best buds in the budget with Type C,I have been researching about getting a nice T...,5.0,Best buds in the budget with Type C I have bee...


In [306]:
df=df[df['Rating']!=3.0]

In [307]:
df.sample(5)


,Review_Title,Reviews,Rating,text
477,"Pathetic product, please don’t buy, boAt has f...",This is a highly overrated product! I’m writin...,1.0,"Pathetic product, please don’t buy, boAt has f..."
1881,BEST SOUND QUALITY IN THIS PRICE RANGE,Got it for 1699/-. Sound quality is pretty goo...,5.0,BEST SOUND QUALITY IN THIS PRICE RANGE Got it ...
123,Just amazing and perfect!,I knew I wanted Boats this time but which one?...,5.0,Just amazing and perfect! I knew I wanted Boat...
1575,Defective product.,Charging case of airdopes was defective from t...,1.0,Defective product. Charging case of airdopes w...
3296,Trust in BOAT,I always had a trust in boat and when i m orde...,4.0,Trust in BOAT I always had a trust in boat and...


In [308]:
def rating_classify(rating):
  if rating>=4:
    return 1
  else:
    return 0

In [309]:
df['label']=df['Rating'].apply(rating_classify)

In [310]:
df.head()

,Review_Title,Reviews,Rating,text,label
0,AWESOME...IN LOVE WITH IT !,Really impressed with the fastest delivery of ...,5.0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1
1,Enjoy while it lasts,This is the second time I am writing this revi...,2.0,Enjoy while it lasts This is the second time I...,0
2,Good,1. Quite good look2. Better noise cancellation...,4.0,Good 1. Quite good look2. Better noise cancell...,1
3,not great,its a good set of ear buds but1. the quality o...,1.0,not great its a good set of ear buds but1. the...,0
4,Best buds in the budget with Type C,I have been researching about getting a nice T...,5.0,Best buds in the budget with Type C I have bee...,1


In [311]:
df.drop(['Review_Title','Reviews','Rating'],axis=1,inplace=True)

In [312]:
df.head()

,text,label
0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1
1,Enjoy while it lasts This is the second time I...,0
2,Good 1. Quite good look2. Better noise cancell...,1
3,not great its a good set of ear buds but1. the...,0
4,Best buds in the budget with Type C I have bee...,1


In [313]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [314]:
port_stem=PorterStemmer()
def text_processing(text):
  text=re.sub('[^a-zA-Z0-9]',' ',text)
  text=text.lower()
  # text=text.split()
  # text = [port_stem.stem(word) for word in text if not word in stopwords.words('english')]
  # text = [port_stem.stem(word) for word in text]
  # text = ' '.join(text)
  return text

In [315]:
df['new_text']=df['text'].apply(text_processing)

In [316]:
df.head()

,text,label,new_text
0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1,awesome in love with it really impressed w...
1,Enjoy while it lasts This is the second time I...,0,enjoy while it lasts this is the second time i...
2,Good 1. Quite good look2. Better noise cancell...,1,good 1 quite good look2 better noise cancell...
3,not great its a good set of ear buds but1. the...,0,not great its a good set of ear buds but1 the...
4,Best buds in the budget with Type C I have bee...,1,best buds in the budget with type c i have bee...


In [317]:
import tensorflow as tf

In [318]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.layers import LSTM,Bidirectional,GRU
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
txt=df['new_text'].to_list()
txt

In [322]:
len(txt)

3104

In [323]:
df.shape

(3104, 3)

In [324]:
y=df['label']

In [325]:
token=Tokenizer()
token.fit_on_texts(txt)

In [ ]:
token.index_word

In [327]:
vocab=len(token.index_word)+1
vocab

9028

In [ ]:
encoded_text=token.texts_to_sequences(txt)
encoded_text

In [329]:
len(encoded_text)

3104

In [330]:
max_length=200
x=pad_sequences(encoded_text,maxlen=max_length,padding='post')


In [331]:
x

array([[ 111,   10,  231, ...,    0,    0,    0],
       [ 687,   76,    3, ...,    0,    0,    0],
       [   9,   77,  245, ...,    0,    0,    0],
       ...,
       [ 151,  224,  137, ...,    0,    0,    0],
       [ 157,   13,  192, ...,    0,    0,    0],
       [  56, 9025,   16, ...,    0,    0,    0]], dtype=int32)

In [332]:
x.shape

(3104, 200)

In [333]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=2,test_size=0.2,stratify=y)

In [406]:
embedding_vector_features=1000
model=Sequential()
model.add(Embedding(vocab,embedding_vector_features,input_length=max_length))
model.add(Dropout(0.1))
# model.add(LSTM(128))
model.add(Bidirectional(LSTM(64)))
# model.add(GRU(64)) 
# model.add(SimpleRNN(64)) 
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [135]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(100)))
 
# model.add(Dropout(0.2))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [407]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [387]:
print(model.summary())

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, 200, 1000)         9028000   
                                                                 
 bidirectional_21 (Bidirecti  (None, 512)              2574336   
 onal)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_27 (Dense)            (None, 1)                 513       
                                                                 
Total params: 11,602,849
Trainable params: 11,602,849
Non-trainable params: 0
_________________________________________________________________
None


In [408]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

Epoch 1/5
78/78 [==============================] - 6s 38ms/step - loss: 0.4321 - accuracy: 0.7910 - val_loss: 0.3060 - val_accuracy: 0.8808
Epoch 2/5
78/78 [==============================] - 2s 28ms/step - loss: 0.2071 - accuracy: 0.9219 - val_loss: 0.2904 - val_accuracy: 0.8889
Epoch 3/5
78/78 [==============================] - 2s 28ms/step - loss: 0.1104 - accuracy: 0.9674 - val_loss: 0.3533 - val_accuracy: 0.8631
Epoch 4/5
78/78 [==============================] - 2s 28ms/step - loss: 0.0711 - accuracy: 0.9803 - val_loss: 0.4054 - val_accuracy: 0.8663
Epoch 5/5
78/78 [==============================] - 2s 28ms/step - loss: 0.0710 - accuracy: 0.9807 - val_loss: 0.3725 - val_accuracy: 0.8824


In [409]:
def encode(x):
  x=token.texts_to_sequences(x)
  x=pad_sequences(x,maxlen=max_length,padding='post')
  return x


In [430]:
x=["the product is good"]

In [431]:
val=model.predict(encode(x))[0][0]
val

0.9145871

In [433]:
if val>0.5:
  print("positive review")
else: 
  print('negative review')

positive review


In [434]:
pred=[]
# def pred(val):
for i in range(len(x_test)):
  val=model.predict(x_test[i].reshape(1,-1))[0][0]
  if val>0.5:
    pred.append(1)
  else:
    pred.append(0)

In [435]:
len(pred)

621

In [436]:
from sklearn.metrics import accuracy_score,confusion_matrix

accuracy_score(y_test,pred)

0.8824476650563607

In [259]:
len(y_test)

621

In [229]:
len(x_test)

621

In [237]:
model.predict(x_test[0].reshape(1,-1))[0][0]

6.159466e-05

In [124]:
# voc_size=5000

In [125]:
# [one_hot(words,voc_size)for words in df['text']] 
# onehot_repr=[one_hot(words,voc_size)for words in df['text'] ] 


In [ ]:
# sent_length=100
# embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
# print(embedded_docs)

In [ ]:
# embedded_docs[1]

In [130]:
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(LSTM(64))
# model.add(Dense(1,activation='sigmoid'))


In [131]:
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
# print(model.summary())

In [ ]:
# len(embedded_docs),df['label'].shape

In [134]:
# import numpy as np
# X_final=np.array(embedded_docs)
# y_final=np.array(df['label'])

In [135]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
# model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=128)

In [147]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(100)))
 
# model.add(Dropout(0.2))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=128)

In [ ]:
# model = Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.1))
# model.add(SimpleRNN(64)) 
# model.add(Dropout(0.1))
# model.add(Dense(1, activation='sigmoid'))

# model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

# model.summary()

In [221]:
# from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, SimpleRNN, GRU

In [ ]:
# model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=128)